In [0]:
!pip install sentence-transformers

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 573kB 6.3MB/s 
     |████████████████████████████████| 3.7MB 16.3MB/s 
     |████████████████████████████████| 1.0MB 27.7MB/s 
     |████████████████████████████████| 890kB 48.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=2c76390237c88982457865f129d1d162344d0b1cf8b3b6412b57e18e181bc294
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=f8e52611021799f7b8dd2887d94719d5ced8cb168f5aff0cc0faffc076fa3370
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sentence-transformers sacremoses


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

lemmatizer = WordNetLemmatizer()
print(tag_map)

def lemmatization(tweet):
  tokens = word_tokenize(tweet)
  tweet = ' '.join([lemmatizer.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(tokens)])
  return tweet

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
defaultdict(<function <lambda> at 0x7f01243e6488>, {'J': 'a', 'V': 'v', 'R': 'r'})


In [0]:
import re
import string
from bs4 import BeautifulSoup
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#for tweet in tweet_data:
def preprocess_tweet(tweet):
    # Clean the training data
    # print(tweet)
    
    #Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
    
    #lowercase
    tweet = tweet.lower()
    
    #remove links
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','',tweet)
    
    #Remove usernames
    tweet = re.sub('@[^\s]+','', tweet)
    
    # replace hashtags by just words
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    
    #Removal of Punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    
    #correct all multiple white spaces to a single white space
    tweet = re.sub('[\s]+', ' ', tweet)
   
    #remove space at the beginning and the end
    tweet = tweet.strip()

    #tweet = ''.join([lemmatizer.lemmatize(w) for w in tweet])
    tweet = lemmatization(tweet)
    
    # print(tweet)
    
    return tweet

In [0]:
from sentence_transformers import SentenceTransformer
import scipy.spatial

embedder = SentenceTransformer('bert-large-nli-mean-tokens') #bert-base-nli-mean-tokens

100%|██████████| 1.24G/1.24G [00:45<00:00, 27.1MB/s]


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import re

usecols = (4,7,8)
train_dataframe = pd.read_table("/content/gdrive/My Drive/task3_training.tsv",usecols=usecols)
test_dataframe = pd.read_table("/content/gdrive/My Drive/task3_validation.tsv",usecols=usecols)

print(train_dataframe.shape)
print(test_dataframe.shape)
print(train_dataframe.head())

(2246, 3)
(560, 3)
  extraction  meddra_code meddra_term
0        NaN          NaN         NaN
1        NaN          NaN         NaN
2        NaN          NaN         NaN
3        NaN          NaN         NaN
4        NaN          NaN         NaN


In [0]:
train_data = train_dataframe.iloc[:, 0]
train_code = train_dataframe.iloc[:, 1]
train_term = train_dataframe.iloc[:, 2]

test_data = test_dataframe.iloc[:, 0]
test_code = test_dataframe.iloc[:, 1]
test_term = test_dataframe.iloc[:, 2]
print(train_code[0])
print(train_data[0])
print(type(train_code[0]))
print(type(train_code[30]))
#print(train_data)

nan
nan
<class 'numpy.float64'>
<class 'numpy.float64'>


In [0]:
extracted_train_term =[] #med term : med code
extracted_train_code =[]
extracted_train_data =[]
for index, data in enumerate(train_term):
  if type(data) == str:
    #print(index, data, train_data[index], train_code[index])
    extracted_train_term.append(data)   #2nd column - expected
    extracted_train_code.append(train_code[index])
    extracted_train_data.append(train_data[index])


extracted_test_data = []
extracted_test_term = [] #med term : med code
extracted_test_code =[]
for index, data in enumerate(test_term):
  if type(data) == str:
    extracted_test_term.append(data) #2nd column - expected
    extracted_test_code.append(test_code[index])
    extracted_test_data.append(test_data[index])

In [0]:
corpus = extracted_train_term + extracted_test_term
corpus_embeddings = embedder.encode(corpus)
# print(corpus)
# print(extracted_train_data)

In [0]:
queries = extracted_train_data + extracted_test_data
query_embeddings = embedder.encode(queries)
# print(extracted_test_term)
# print(queries)


In [0]:
print(corpus)
print(queries)

['vision loss', 'withdrawal syndrome', 'nauseous', 'inability to orgasm', 'weight gain', 'weight gain', 'headache', 'condition worsened', 'appetite suppressed', 'pain', 'drug toxicity', 'feeling unwell', 'confusion', 'dizziness', 'spaced out', 'increasd skin sensitivity', 'rash', 'feeling strange', 'sleeplessness', 'headache', 'hair loss', 'hair loss', 'sleeplessness', 'calcium deficiency', 'vitamin d deficiency', 'depression', 'weight gain', 'weight gain', 'liver damage', 'allergic reaction', 'near death experience', 'mixed manic depressive episode', 'feeling high', 'withdrawal syndrome', 'pain', 'pain in heel', 'pain', 'arm injury', 'migraine', 'vomiting', 'illogical thinking', 'slowed thinking', 'feeling irritated', 'felt like a zombie', 'somnolence', 'ache', 'flu like symptoms', 'shivers', 'sweating', 'rage', 'ache', 'back pain', 'fluid retention', 'swelling of legs', 'drug withdrawal syndrome', 'electric shock sensation', 'lack of feeling emotions', 'lack of feeling emotions', 'an

In [0]:
actual_adr = extracted_train_term + extracted_test_term   # corpus
actual_code = extracted_train_code + extracted_test_code

In [0]:
def find_distance(queries, query_embeddings, corpus, corpus_embeddings, closest_n,\
                   actual_adr, actual_adr_code, corpus_train_code):
  index = 0
  correct = 0
  file = open(r"/content/gdrive/My Drive/results_task3.txt","w") 
  for adr, query_embedding in zip(queries, query_embeddings):
      distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

      results = zip(range(len(distances)), distances)
      results = sorted(results, key=lambda x: x[1])

      print("\n\n======================\n\n")
      file.write("\n\n======================\n\n")
      print("Extracted ADR from tweet: ",adr)
      file.write("Extracted ADR from tweet: "+ adr)

      for idx, distance in results[0:closest_n]:
          print("\nMatched ADR term in corpus:", corpus[idx].strip(), '\nMatched ADR code', int(actual_adr_code[idx]),\
                "(Score: %.4f)" % (1-distance))
          file.write("\nMatched ADR term in corpus:"+ corpus[idx].strip()+ '\nMatched ADR code'+ str(int(actual_adr_code[idx]))+\
                "(Score: %.4f)" % (1-distance))
          

          print('\nExpected ADR term : ', actual_adr[index], '\nExpected ADR code : ', int(actual_adr_code[index]))
          file.write('\nExpected ADR term : '+ actual_adr[index]+ '\nExpected ADR code : '+ str(int(actual_adr_code[index])))


          if(actual_adr[index].strip() == corpus[idx].strip()):
            correct += 1
            print("\nResult: Correct")
            file.write("\nResult: Correct")
          else:
            print("\nResult: Wrong")
            file.write("\nResult: Wrong")
          index += 1
  print('\nTotal Extracted ADR Evaluated: ', len(actual_adr) ,' Correctly Identified: ', correct)
  print('Accuracy', (correct/float(len(actual_adr)))*100)
  file.write('\nTotal Extracted ADR Evaluated: '+ str(len(actual_adr)) +' Correctly Identified: '+ str(correct))
  file.write('Accuracy: '+ str((correct/float(len(actual_adr)))*100))
  file.close()

In [0]:
closest_n = 1
find_distance(queries, query_embeddings, corpus, corpus_embeddings, closest_n,\
              actual_adr, actual_code, extracted_train_code)

Streaming output truncated to the last 5000 lines.


Extracted ADR from tweet:  tendon rupture

Matched ADR term in corpus: tendon rupture 
Matched ADR code 10043248 (Score: 1.0000)

Expected ADR term :  tendon rupture 
Expected ADR code :  10043248

Result: Correct




Extracted ADR from tweet:  withdrawal

Matched ADR term in corpus: withdrawn 
Matched ADR code 10048013 (Score: 0.9390)

Expected ADR term :  withdrawal syndrome 
Expected ADR code :  10048010

Result: Wrong




Extracted ADR from tweet:  emotional mess

Matched ADR term in corpus: emotional distress 
Matched ADR code 10049119 (Score: 0.9564)

Expected ADR term :  emotional distress 
Expected ADR code :  10049119

Result: Correct




Extracted ADR from tweet:  not remember

Matched ADR term in corpus: lack of feeling emotions 
Matched ADR code 10071175 (Score: 0.8682)

Expected ADR term :  memory loss 
Expected ADR code :  10027176

Result: Wrong




Extracted ADR from tweet:  memory lapse

Matched ADR term in corpus: m